In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import seaborn as sns
import collections

%reload_ext lab_black

# Diameter of the graph

In [ ]:
actors_agg_adj = np.load("sparse_agg_actor_adj.npy")
actors_graph = nx.from_numpy_matrix(actors_agg_adj)

In [ ]:
print(f"The diameter of the graph is: {nx.diameter(actors_graph)}")

# Sparsity of the graph

#### HeatMap

In [ ]:
# sns.heatmap(actors_agg_adj)

### Sparsity 

In [ ]:
plt.figure(figsize=(15, 10))
plt.spy(actors_agg_adj)
plt.title("Sparsity of Adjj Matrix")
plt.savefig("plots/sparsity_adjj.png")
plt.show()

# Connected components

In [ ]:
connected_components = nx.number_connected_components(actors_graph)
connected_components

# Degree Distribution

In [ ]:
degree_sequence = sorted(
    [d for n, d in actors_graph.degree()], reverse=True
)  # degree sequence
# print "Degree sequence", degree_sequence
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

plt.figure(figsize=(15, 10))
plt.bar(deg, cnt, width=0.80, color="b")
plt.title("Degree Distribution")
plt.xlabel("Degree")
plt.ylabel("Frequency")
plt.grid(True)
plt.savefig("plots/degree_distribution.png")
plt.show()

# Spectrum

In [ ]:
lamb_comb = nx.laplacian_spectrum(actors_graph)
lamb_norm = nx.normalized_laplacian_spectrum(actors_graph)

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(121)
plt.plot(lamb_comb)
plt.xlabel("Index")
plt.ylabel("Eigenvalue")
plt.title("Eigenvalues $L_{comb}$")
plt.subplot(122)
plt.plot(lamb_norm)
plt.xlabel("Index")
plt.ylabel("Eigenvalue")
plt.title("Eigenvalues $L_{norm}$")
plt.savefig("plots/spectrum")
plt.show()

# Average Degree

In [ ]:
average_degree = nx.average_degree_connectivity(actors_graph)
# average_degree
dict(list(average_degree.items())[0:10])

# Plot the Graph

In [ ]:
nx.draw(actors_graph)

In [ ]:
nx.draw_networkx(actors_graph)

In [ ]:
# # Graph-tool
# g = Graph()
# g.add_edge_list(np.transpose(actors_agg_adj[100:200, 100:200].nonzero()))

In [ ]:
# gt.draw.graph_draw(g, fmt="png", output_size=(600, 600))

In [ ]:
Gc_actors = max(nx.connected_component_subgraphs(actors_graph), key=len)
nb_edges_gc = Gc_actors.number_of_edges()
nb_nodes_gc = Gc_actors.number_of_nodes()
coords_Gc = nx.spring_layout(Gc_actors, k=0.03)

In [ ]:
plt.figure(figsize=(28, 7))
im = nx.draw_networkx_nodes(
    Gc_actors, coords_Gc, node_size=10, cmap="tab20", vmin=0, vmax=20
)
nx.draw_networkx_edges(Gc_actors, coords_Gc, alpha=0.1, width=0.7)
plt.title("Graph, with {} edges and {} nodes".format(nb_edges_gc, nb_nodes_gc))
plt.colorbar(im)
plt.savefig("plots/graph")
plt.show()